In [2]:
import pandas as pd
import numpy as np 
import pickle

df = pd.read_csv('./data/train_triplets.txt', sep='\t', names=['user', 'song', 'plays'])
df2 = pd.read_csv('./data/msd-topMAGD-genreAssignment.cls', sep='\t', names=['track', 'genre'])
df3 = pd.read_csv('./data/unique_tracks.txt', sep='<SEP>', names=['track', 'song', 'artist', 'album'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys


In [3]:
print(df.head())
print(df2.head())
print(df3.head())

                                       user                song  plays
0  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAKIMP12A8C130995      1
1  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAPDEY12A81C210A9      1
2  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBBMDR12A8C13253B      2
3  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBFNSP12AF72A0E22      1
4  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBFOVM12A58A7D494      1
                track     genre
0  TRAAAAK128F9318786  Pop_Rock
1  TRAAAAV128F421A322  Pop_Rock
2  TRAAAAW128F429D538       Rap
3  TRAAABD128F429CF47  Pop_Rock
4  TRAAACV128F423E09E  Pop_Rock
                track                song            artist              album
0  TRMMMYQ128F932D901  SOQMMHC12AB0180CB8  Faster Pussy cat       Silent Night
1  TRMMMKD128F425225D  SOVFVAK12A8C1350D9  Karkkiautomaatti        Tanssi vaan
2  TRMMMRX128F93187D9  SOGTUKN12AB017F4F1    Hudson Mohawke  No One Could Ever
3  TRMMMCH128F425532C  SOBNYVR12A8C13558C       Yerba Brava     Si

In [4]:
dftemp = df.merge(df3, on='song', how='left')
dftemp2 = dftemp.merge(df2, on='track')
print(dftemp2.head())

                                       user                song  plays  \
0  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBFNSP12AF72A0E22      1   
1  0f7d7d02867e87f47701c70ceb6b994c11b4db85  SOBFNSP12AF72A0E22      1   
2  f926327f2c99580a4aa414d1cfc0439f45a929bf  SOBFNSP12AF72A0E22      5   
3  85812d920b63e8a278f79f4c68cb89f4960efa75  SOBFNSP12AF72A0E22      1   
4  591e295328be6bfb1f8ba124aaca0b7a85347d66  SOBFNSP12AF72A0E22     10   

                track      artist                  album     genre  
0  TRYQMNI128F147C1C7  Josh Rouse  Under Cold Blue Stars  Pop_Rock  
1  TRYQMNI128F147C1C7  Josh Rouse  Under Cold Blue Stars  Pop_Rock  
2  TRYQMNI128F147C1C7  Josh Rouse  Under Cold Blue Stars  Pop_Rock  
3  TRYQMNI128F147C1C7  Josh Rouse  Under Cold Blue Stars  Pop_Rock  
4  TRYQMNI128F147C1C7  Josh Rouse  Under Cold Blue Stars  Pop_Rock  


In [5]:
print(dftemp2.isnull().sum())

user      0
song      0
plays     0
track     0
artist    0
album     0
genre     0
dtype: int64


In [6]:
df = dftemp2
genre_dict = {}
with open('./data/genre_dict.pickle', 'rb') as f:
    genre_dict = pickle.load(f)
# Take num_users of the most active users in the dataset
num_users = 1000
users = df.groupby('user').sum().sort_values('plays', ascending=False).head(num_users).reset_index().user
df = df.loc[df['user'].isin(users)]
print(len(list(df.track.unique())))
df.to_hdf('./data/listens_with_genres', 'df')

50689


In [7]:
piv = pd.pivot_table(df, index='user', columns='song', values='plays', fill_value=0)
genres = [genre_dict[x] for x in list(piv.columns)]

In [8]:
print(genres[0:100])

['Pop_Rock', 'Rap', 'Pop_Rock', 'Rap', 'Pop_Rock', 'Pop_Rock', 'Rap', 'Pop_Rock', 'Pop_Rock', 'Jazz', 'Pop_Rock', 'Pop_Rock', 'Electronic', 'Pop_Rock', 'Pop_Rock', 'Pop_Rock', 'Pop_Rock', 'Pop_Rock', 'Rap', 'Electronic', 'Pop_Rock', 'Jazz', 'Pop_Rock', 'Pop_Rock', 'Pop_Rock', 'RnB', 'Pop_Rock', 'Pop_Rock', 'Rap', 'Pop_Rock', 'RnB', 'Jazz', 'Pop_Rock', 'Pop_Rock', 'Folk', 'Pop_Rock', 'Pop_Rock', 'Country', 'Pop_Rock', 'Electronic', 'Pop_Rock', 'Pop_Rock', 'Pop_Rock', 'Electronic', 'Electronic', 'Pop_Rock', 'Electronic', 'Pop_Rock', 'Pop_Rock', 'Pop_Rock', 'Pop_Rock', 'Electronic', 'RnB', 'Pop_Rock', 'Latin', 'Vocal', 'Pop_Rock', 'Pop_Rock', 'Pop_Rock', 'RnB', 'Pop_Rock', 'Electronic', 'Pop_Rock', 'Pop_Rock', 'Electronic', 'Pop_Rock', 'Rap', 'Electronic', 'Pop_Rock', 'Pop_Rock', 'Pop_Rock', 'Rap', 'Pop_Rock', 'Pop_Rock', 'Pop_Rock', 'Pop_Rock', 'Jazz', 'Pop_Rock', 'Pop_Rock', 'Pop_Rock', 'Pop_Rock', 'Country', 'Pop_Rock', 'Pop_Rock', 'Rap', 'Pop_Rock', 'Pop_Rock', 'Pop_Rock', 'Electronic

In [13]:
import pickle
with open('./data/genre_list.pickle', 'wb') as f:
    pickle.dump(genres, f)